In [0]:
!pip install --upgrade keras
!pip install gdown
!pip install tensorflow-gpu

In [0]:
%cd /content/drive/My Drive/person_detection/keras-retinanet

!pip install .
!python setup.py build_ext --inplace
%cd ../../../../../content/sample_data/

In [0]:
!git clone https://github.com/Yannick947/WiderPerson.git
%cd /../..

In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil
import time
import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.bin import evaluate


%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


In [0]:
annot_path = '/content/drive/My Drive/person_detection/WiderPerson/Annotations'
images_path = '/content/drive/My Drive/person_detection/WiderPerson/Images'
images_path_sample = '/content/sample_data/WiderPerson/Images'
annot_csv_path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations_sampledata.csv'
keras_path = '/content/drive/My Drive/person_detection/keras-retinanet'
#Remove classes which shall be left out in the csv
classes_ids = {1:'pedestrian',
               2:'riders',
               3:'partially-visible',
               5:'crowd'}


# Training



In [0]:
classes = pd.read_csv('/content/drive/My Drive/person_detection/keras-retinanet/classes.csv', header=None)

annot_test = pd.read_csv('/content/drive/My Drive/person_detection/keras-retinanet/annot_test.csv',
                    names=['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'label'])

labels_to_names = classes.T.loc[0].to_dict()
annot_test.head()


# Testing


In [0]:
# Convert to inference Model 
# tf1.disable_v2_behavior()

model_path = '/content/drive/My Drive/person_detection/keras-retinanet/snapshots/resnet50_csv_15.h5'
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

In [0]:
%cd /content/drive/My Drive/person_detection/keras-retinanet
for i in range(1):
  for ii in range(3):
    iou = -i / 10 + 0.5
    thresh = ii / 20 + 0.35
    print(thresh, iou)
    !retinanet-evaluate csv annot_test.csv classes.csv snapshots/resnet50_csv_19.h5 --convert-model --score-threshold=$thresh  --iou-threshold=$iou
%cd ../../../../..

In [0]:
def show_image_objects(image_row):
  #TODO: make multiple boxes possible

  box = [image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]

  image = read_image_bgr(image_row.image_name)

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  draw_box(draw, box, color=(255, 255, 0))

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [0]:
show_image_objects(annot_test.iloc[500])

In [0]:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels

In [0]:
THRES_SCORE = 0.55

def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        break

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(image, b, caption)


In [0]:
def filter_images_by_size(df, max_size=4000, min_size=440):
  start_size = len(os.listdir(images_path_sample))
  for image_name in os.listdir(images_path_sample):
    try: 
      image = read_image_bgr(images_path_sample + '/' + image_name)
      if (min(image.shape[0:2]) < min_size) or (max(image.shape[0:2]) > max_size):
        df = df[~df.image_name.str.contains(image_name)]
    except: 
      print('Image not in dataset. Name of file: ', image_name)
  print('final df shape: ', df.shape)
  print('Removed ', df.image_name.nunique() - start_size, 'images')
  return df

df = pd.read_csv(annot_csv_path,
                  header=None,
                  names=['image_name', 'x1', 'y1', 'x2', 'y2', 'label'])

df_filtered = filter_images_by_size(df, min_size=600, max_size = 3000)
df_filtered.to_csv('annot_filtered_600_3000.csv', header=None, index=None)


In [0]:
%cd ../../../../
#Splitting into train and test data
train_df, test_df = train_test_split(
  pd.read_csv('/content/drive/My Drive/person_detection/keras-retinanet/annot_filtered_450_2000.csv',
              header=None,
              names=['image_name', 'x1', 'y1', 'x2', 'y2', 'label']), 
              test_size=0.15,
              random_state=RANDOM_SEED
  )

train_df.to_csv(keras_path + '/annot_train_filtered_450_2000.csv', header=None, index=None)
test_df.to_csv(keras_path + '/annot_test_filtered_450_2000.csv', header=None, index=None)


In [0]:
shapes_x = list()
shapes_y = list()
for i in range(1500):
  path = annot_test.iloc[i].image_name
  image = read_image_bgr(path)
  shapes_x.append(image.shape[0])
  shapes_y.append(image.shape[1])

print('average in x', sum(shapes_x) / len(shapes_x))
print('average in y', sum(shapes_y) / len(shapes_y))
print('std in x', np.std(shapes_x), '\nstd in y', np.std(shapes_y))




In [0]:
def show_detected_objects(image_row):
  img_path = image_row.image_name
  # img_path = '/content/drive/My Drive/person_detection/WiderPerson/WIN_20200405_11_04_03_Pro.jpg'
  image = read_image_bgr(img_path)

  boxes, scores, labels = predict(image)

  for i, score in enumerate(scores[0]): 
    if score > THRES_SCORE:
      print('row ', image_row.image_name, 'score ', scores[0, i])
  
  if all(i < THRES_SCORE for i in scores[0]):
    return

  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  true_box = [image_row.x_min, image_row.y_min, 
              image_row.x_max, image_row.y_max]
  
  #draw_box(draw, true_box, color=(255, 255, 0))

  draw_detections(draw, boxes, scores, labels)

  plt.axis('off')
  plt.imshow(draw)
  plt.show()

for i in range(1,501, 50): 
  show_detected_objects(annot_test.iloc[i])

In [0]:
def replace_annoation_folder(new_path, csv_annot_path, img_path):
  '''Start this function from folder root, otherwise wont work properly'''

  df_annot = pd.read_csv('/content/annotations_sample.csv',
                         names=['name', 'x1', 'y1', 'x2', 'y2', 'label'])
  df_annot = df_annot.reset_index().drop(0).drop(columns='index')
  df_annot['name'] = df_annot['name'].str.replace(img_path, new_path)
  df_annot = df_annot.dropna()
  df_annot.to_csv(csv_annot_path + '/annotations_sampledata.csv',
                  index=None, header=None)
  return

new_path = '/content/sample_data/WiderPerson/Images'
path_to_csv_annot = '/content/drive/My Drive/person_detection/keras-retinanet'
old_path = '/content/drive/My Drive/PersonDetection/WiderPerson/Images'
replace_annoation_folder(new_path, path_to_csv_annot, old_path)

In [0]:
#create indexing csv file
with open('classes.csv', newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for index, key in enumerate(classes_ids.keys()):
    csv_writer.writerow([str(classes_ids[key]), index])
    
#Check for wrongly annotaded bounding boxes
def check_bb(path):
  colnames = ['filename', 'x1', 'y1', 'x2', 'y2', 'class_label']
  df = pd.read_csv(path, names=colnames)
  df_new = df.loc[(df.x1 < df.x2) & (df.y1 < df.y2), : ]
  print ('Reduces shape from ', df.shape, 'to ', df_new.shape)
  return df_new

df = check_bb(path)
